In [3]:
import pandas as pd
import csv

In [4]:
f = open('badwords/badwords.txt', 'r', encoding='UTF-8')

badwords1 = f.read()

badwords1 = ''.join(badwords1.split())
badwords1 = ''.join(badwords1.split('"'))
badwords1 = badwords1.split(',')

f.close()

f = open('badwords/badwords2.txt', 'r', encoding='cp949')

badwords2 = f.read()

badwords2 = ''.join(badwords2.split())
badwords2 = ''.join(badwords2.split('"'))
badwords2 = badwords2.split(',')

f.close()

bad_words = badwords1 + badwords2
bad_words = list(set(bad_words))

print(len(bad_words))
bad_words.sort()
print(bad_words[123])

658
개쌔끼


In [5]:
data1 = pd.read_csv('badwords/reply_data1.csv', sep='\t')

data1 = data1.replace(0, 2)
data1 = data1.replace(1, 0)
data1 = data1.replace(2, 1)

data1 = data1.rename(columns={'content':'comment'})

data1[data1['lable'].isnull()]
data1['lable'] = pd.to_numeric(data1['lable'], errors='coerce')
data1 = data1.dropna(subset=['lable'])
data1['lable'] = data1['lable'].astype(int)

In [6]:
f = open('badwords/reply_data2.txt', 'r', encoding='UTF-8')

data2_list = f.read()

f.close()

data2_list = ''.join(data2_list.split('|'))
data2_list = data2_list.split('\n')

lable = []
for i in range(0, len(data2_list)):
    lable.append(data2_list[i][-1])
    data2_list[i] = data2_list[i][:-1]

data2 = pd.DataFrame([x for x in zip(data2_list,lable)])

data2.columns = ['comment', 'lable']

In [7]:
data3 = pd.read_csv('badwords/reply_data3.txt', sep = "\t", engine='python', encoding = "utf-8")
data3 = data3.drop(['id'], axis=1)

data3 = data3.replace(1, 2)
data3 = data3.replace(0, 1)
data3 = data3.replace(2, 0)

data3.columns = ['comment', 'lable']

In [8]:
data4 = pd.read_csv('badwords/reply_data4.tsv', sep = "\t", engine='python', encoding = "utf-8")

data4 = data4.drop(['contain_gender_bias', 'bias'], axis=1)
data4 = data4.replace('hate', 1)
data4 = data4.replace('none', 0)

data4.columns = ['comment', 'lable']

In [9]:
reply_data = pd.concat([data1, data2, data4])
data3 = data3.sample(n=40000)

reply_data = pd.concat([reply_data, data3])
reply_data

reply_data = reply_data.replace('offensive', 1)
reply_data['lable'] = reply_data['lable'].astype(int)

In [10]:
import warnings
warnings.filterwarnings(action = 'ignore')
from konlpy.tag import Okt # 형태소 분석에 사용할 konlpy 패키지의 Okt 클래스를 임포트하고 okt 객체를 생성
okt = Okt()

train_df = reply_data

# 댓글이 있는 항목만 담기(빈 댓글 삭제)
# text 컬럼이 non-null인 샘플만 train_df에 다시 저장
train_df = train_df[train_df['comment'].notnull()]

# 수정된 train_df의 정보를 다시 확인
print(train_df.info())

# 분류 클래스의 구성을 확인
print(train_df['lable'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63642 entries, 0 to 71363
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  63642 non-null  object
 1   lable    63642 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 1.2+ MB
None
0    32290
1    31352
Name: lable, dtype: int64


In [11]:
# 한글 외 문자 제거(옵션)
import re # 정규식을 사용하기 위해 re 모듈을 임포트
# ‘ㄱ ~‘힣’까지의 문자를 제외한 나머지는 공백으로 치환, 영문: a-z| A-Z
train_df['comment'] = train_df['comment'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
print(train_df.head())

# Train용 데이터셋의 정보를 재확인
print(train_df.info())
text = train_df['comment'] # 시리즈 객체로 저장
score = train_df['lable']

                                             comment  lable
0  이종석 한효주 나오는 드라마 이후로 드라마 안봤다   년전인가  좀 신선했었지  근...      1
1                       씨바알 노무노무 술프노  오늘 저녁은 꽂등심이다ㅠㅜ      1
2                                           짱깨 꺼라ㅡ패쓰      1
3  그들의 사생활   고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...      0
4  아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...      0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 63642 entries, 0 to 71363
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  63642 non-null  object
 1   lable    63642 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 1.2+ MB
None


In [12]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

50913 50913 12729 12729


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
print(tfv_train_x)

  (0, 22181)	0.47646679687526383
  (0, 16591)	0.569052314104817
  (0, 16580)	0.3611067420896998
  (0, 8411)	0.40369945255291395
  (0, 3020)	0.3946992882096689
  (1, 49942)	0.15337995764541415
  (1, 47607)	0.1450761518041755
  (1, 47243)	0.1581797799478359
  (1, 45063)	0.19215739487848205
  (1, 40455)	0.23897839952258304
  (1, 39659)	0.23387616688733415
  (1, 39657)	0.17744245906852754
  (1, 38217)	0.17904769685212685
  (1, 35685)	0.21385835711170045
  (1, 35274)	0.18690811834111867
  (1, 34851)	0.10304152948172575
  (1, 33407)	0.22618264575964284
  (1, 33096)	0.07340276376274597
  (1, 32890)	0.09047117845341571
  (1, 31979)	0.21202816075200132
  (1, 31953)	0.13379465425585896
  (1, 29221)	0.1696384775908569
  (1, 28601)	0.06538859880517656
  (1, 27566)	0.19999891165537845
  (1, 27557)	0.16278671708763198
  :	:
  (50911, 25480)	1.0
  (50912, 48999)	0.196008549938244
  (50912, 47188)	0.13341325345616506
  (50912, 44178)	0.2714303692296781
  (50912, 44177)	0.17765402016587678
  (50912, 44

In [14]:
from sklearn.linear_model import LogisticRegression # 이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 15} 0.8036650757174003


In [15]:
tfv_test_x = tfv.transform(test_x)
# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)
test_predict = grid_cv.best_estimator_.predict(tfv_test_x)
from sklearn.metrics import accuracy_score
print('감성 분류 모델의 정확도 : ',round(accuracy_score(test_y, test_predict), 3))

감성 분류 모델의 정확도 :  0.816


In [39]:
from sklearn.metrics import classification_report
print(classification_report(test_y, test_predict, target_names=['class 0', 'class 1']))

              precision    recall  f1-score   support

     class 0       0.82      0.82      0.82      6506
     class 1       0.81      0.81      0.81      6223

    accuracy                           0.82     12729
   macro avg       0.82      0.82      0.82     12729
weighted avg       0.82      0.82      0.82     12729



In [75]:
input_text = '재미있다'

#입력 텍스트에 대한 전처리 수행
input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
input_text = [" ".join(input_text)]

# 입력 텍스트의 피처 벡터화
st_tfidf = tfv.transform(input_text)

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = grid_cv.best_estimator_.predict(st_tfidf)

#예측 결과 출력
if(st_predict == 1):
    print("부정")
else:
    print("긍정")

긍정


In [106]:
taxi = pd.read_excel('data/hanmunchul_taxi.xlsx')

In [108]:
hate = 0
error = 0

for i in range(0, len(taxi)):
    try:
        if (i % 500 == 0):
            print(input_text, st_tfidf)
        input_text = taxi['comment'][i]

        #입력 텍스트에 대한 전처리 수행
        input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(input_text))
        input_text = [" ".join(input_text)]

        # 입력 텍스트의 피처 벡터화
        st_tfidf = tfv.transform(input_text)

        # 최적 감성 분석 모델에 적용하여 감성 분석 평가
        st_predict = grid_cv.best_estimator_.predict(st_tfidf)

        #예측 결과 출력
        if(st_predict == 1):
            print("혐오 : ", input_text)
            hate += 1
        else:
            continue
    except:
        error += 1
        print("오류")
        pass

print("혐오 표현의 비율 : ", hate / (len(taxi) - error))

['삼가 고인이 명복을 빕니다']   (0, 34851)	0.09493981062476838
  (0, 33800)	0.32918066227192017
  (0, 33574)	0.13267292453104093
  (0, 22222)	0.3648814735891005
  (0, 22221)	0.35991869706277757
  (0, 21319)	0.32770209299183195
  (0, 16519)	0.3255771939757112
  (0, 16518)	0.3229010629245032
  (0, 3565)	0.41679772540328786
  (0, 3562)	0.33312137345081805
혐오 :  ['너도 꼭 이만큼만 당해라 나쁜시키']
혐오 :  ['저런 택시 족쳐야 된다 인간미가 없다 정말 쓰래기다 인간쓰래기']
혐오 :  ['택시기사 이사람 사람이라고 해야합니까 사람아닙니다 발암물질 정도밖에는 암적인존재 으휴']
혐오 :  ['저 살인마놈 신상공개 급하다 가족들 전부 이 땅에서 욕 처먹고 쪽팔려가 못살게']
혐오 :  ['참나 지가 의사도 아니고 뭣도 아닌 택시기사면서 환자 상태를 지가 왜 판단해 저 인간 진짜 본인 혹은 자기 부모 가족이 위급상황때 똑같이 누가 길막해봐야 정신차림 자기가 책임진다고 한게 꼴랑 년 개월로 벌 받았다고 생각하면 참 오산이다']
혐오 :  ['걍 응급차지나갈때 무조건 길터줘야하는 법안 만들어야됨']
혐오 :  ['아니 영상 제일 앞에 블랙박스 원본 영상을 보여주고 시작하라니까 편집 진짜 개지랄같이 하네 정말']
혐오 :  ['저 택시 기사는 평생운전하지마라 못배운 ㅅㄲ']
혐오 :  ['못배웠으면 적어도 사람은 죽이지 말아야지']
혐오 :  ['구급차 를 양보해야 돼는이유는 나 자신 처럼 아주 중요해합니다']
혐오 :  ['부고소식만 알려주시면 됩니다 쓰레기는 아무대나 버릴 순 없잖아요']
혐오 :  ['피융신 아는척 깝치다가 재입대 한꼴이쥬']
혐오 :  ['그 택시 기사 지가 뭐 산전수전 다 겪어 본 

In [102]:
hate / ((len(taxi)) - error)

0.3965919747790171

In [90]:
print(bad_words)

['10새', '10새기', '10새리', '10세리', '10쉐이', '10쉑', '10스', '10쌔', '10쌔기', '10쎄', '10알', '10창', '10탱', '18', '18것', '18넘', '18년', '18노', '18놈', '18뇬', '18럼', '18롬', '18새', '18새끼', '18색', '18세끼', '18세리', '18섹', '18쉑', '18스', '18아', 'G스팟', 'X끼', 'ass', 'bitch', 'bogi', 'boji', 'bozi', 'damm', 'fuck', 'g랄', 'jaji', 'jazi', 'jot', 'oral', 'sex', 'shit', 'shutup', 'sibal', 'sipal', 'si발', 'ssiba', 'suck', 'tlqkf', 'wtf', 'zot', 'z랄', 'ㄱㅐ', 'ㄲㅈ', 'ㄲㅏ', 'ㄲㅑ', 'ㄲㅣ', 'ㅁㅊ', 'ㅂㅅ', 'ㅄ', 'ㅅㅂ', 'ㅅㅂㄹㅁ', 'ㅅㅍ', 'ㅅㅐ', 'ㅆ1발', 'ㅆㅂㄹㅁ', 'ㅆㅍ', 'ㅆㅣ', 'ㅆ앙', 'ㅈㄹ', 'ㅍㅏ', 'ㅗ', '凸', '간나', '갈보', '갈보년', '강간', '강아지', '같은년', '같은뇬', '개가튼넘', '개같네', '개같은', '개구라', '개년', '개놈', '개뇬', '개대중', '개독', '개돼중', '개랄', '개보지', '개뻥', '개뿔', '개새', '개새기', '개새끼', '개새키', '개색기', '개색끼', '개색키', '개색히', '개섀끼', '개세', '개세끼', '개세이', '개소리', '개쇳기', '개수작', '개쉐', '개쉐리', '개쉐이', '개쉑', '개쉽', '개스끼', '개시키', '개십새기', '개십새끼', '개쌔끼', '개쐑', '개쑈', '개씹', '개아들', '개자슥', '개자식', '개자지', '개접', '개좆', '개좌식', '개처럼', '개허접', '걔새', '걔수작', '걔시끼', '걔시키', '걔썌', '거시기', '걸레', '게색기', '게색

In [96]:
bad_words_count = 0

for i in range(0, len(taxi)):
    input_text = taxi['comment'][i]

    #입력 텍스트에 대한 전처리 수행
    input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(input_text))
    input_text = [" ".join(input_text)]

    if(any(bad_word in str(input_text) for bad_word in bad_words)):
        print(input_text)
        bad_words_count += 1

bad_words_count

['우리나라는 나쁜놈들이 잘사는나라예요 ㅡ오죽하면 돈에눈이뒤집어져가지고 응급환자를태운차를 가로막는 택시기사놈이 있겠어요 저는 잘못없으니 사람이죽어도 괜찮다는 사고방식이 싹텄겠어요 그래 택시기사같으놈들아 너희들은 꼭 그 배 배 고통받을테니 내가 그법을안다 이놈들아 이 멍청놈탴시기사같놈들아 법이 있던없든 한치에 오차도없이 지은대로 받는다는것을 기억하라 인과응보는자업자득 이것 꼭 기억해라 혼자 몰래 짖는죄 꼭 벌받는다 임마 배 배 배 곹 ㄷㅇ 받는다']
['아니 영상 제일 앞에 블랙박스 원본 영상을 보여주고 시작하라니까 편집 진짜 개지랄같이 하네 정말']
['아우 진짜 저 쌍놈은 지금 봐도 화가나네']
['와 택시기사 ㅁㅊ네 저건 책임진다는대 책임안주면 너 나중에 지옥간다 그리고 지옥에서 용함애 빠진다']
['와진짜미친놈이네']
['살인자네 고의로 죽였구만 ㅁㅊㄸㄹ이']
['저런택시기사 ㅆㅂ세끼네 진짜 아우보는내가 화가 치밀어 오르네']
['년 개월형 살고 대한민국에서 꺼져라 알겠냐']
['전문가도 아니면서 저 상황을 본인이 판단하고 사람의 생명을 저렇게 쉽게 책임을 질 수 있다고 말하는 것이 말이 되나 니가 먼데 책임진다카노 ㅋㅋ 택시 기사 닌 진짜 미친놈이다 돌아가신 분이 택시 기사 니 때문인지 아닌지 알 수 없지만 기사 닌 누군가의 부모님이 치료받을 수 있는 정말 아까운 시간을 날려버린 놈이다 하 사람맞나 니도 누군가의 자식이잖아 누군가의 소중한 부모님 동생 누나 친구 형이 구급차로 이동 중이잖아 도대체 왜어ㅐ왜왜왜어애어ㅑ']
['재좀죽여주실분없나요 화가 존나치민다 진짜택시 개샡기']
['이 택레기 쉐끼 어찌 됐나요']
['최대한 고통스럽게 갔으면 좋겠다 미친놈이']
['죽으면 책임 죽으면 돌아 올 수가 없는데 지가 무슨 수로 책임을 지는데 법으로 구급차 고의적으로 막아서서 막장 부리는 놈들 차로 밀어버리든가 아구창 해머로 쳐서 기절시켜도 가능하게 해라']
['와 내가 저 상황이면 응급차에서 내려서 택시 운전사 끌어 내리고 생 지랄 했을텐데 택시운전사 미친넘'

4834

In [99]:
print("욕설이 포함된 댓글 : ", round(bad_words_count / len(taxi) * 100, 3), "%")
print("모델이 혐오 표현으로 분류한 댓글 : ", round(hate / len(taxi) * 100, 3), "%")

욕설이 포함된 댓글 :  9.645 %
모델이 혐오 표현으로 분류한 댓글 :  39.658 %
